In [3]:
"""
Блок 4. Оптимизация через подзапросы
"""

import libs.db_connection

# Пример большого количества джоинов
sql1 = """explain (analyze) 
SELECT * 
FROM task31 t1
left join 
task32 t2 on t2.link2 = t1.link1
left join task33 t3 on t3.link3 = t2.link2
left join task34 t4 on t4.link4 = t2.link2
left join task37 t7 on t7.link7 = t2.link2
left join task38 t8 on t8.link8 = t2.link2
left join task39 t9 on t9.link9 = t2.link2
left join task35 t5 on t5.link5 = t1.link1
left join task36 t6 on t6.link6 = t1.link1
left join task310 t10 on t10.link10 = t1.link1;"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])



[RealDictRow([('QUERY PLAN',
               'Gather  (cost=364308.09..644474.21 rows=1000000 width=544) (actual time=13937.530..16930.408 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Hash Left Join  (cost=363308.09..543474.21 rows=416667 width=544) (actual time=13841.673..14280.439 rows=333333 loops=3)')]),
 RealDictRow([('QUERY PLAN', '        Hash Cond: (t1.link1 = t10.link10)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Parallel Hash Left Join  (cost=339246.09..459145.59 rows=416667 width=490) (actual time=9639.109..10064.751 rows=333333 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '              Hash Cond: (t1.link1 = t6.link6)')]),
 RealDictRow([('QUERY PLAN',
               '              ->  Parallel Hash Left Join  (cost=315181.09..380509.97 rows=416667 width=435) (actual time=7919.660..8239.927 

In [8]:
# Промежуточная материализация

sql1 = """create table task3_materialized as 
select t2.* 
from task32 t2 
left join task33 t3 on t3.link3 = t2.link2
left join task34 t4 on t4.link4 = t2.link2
left join task37 t7 on t7.link7 = t2.link2
left join task38 t8 on t8.link8 = t2.link2;"""

libs.db_connection.execute(sql1)


In [9]:
# Переписанный запрос
sql1 = """explain (analyze) 
SELECT * 
FROM task31 t1
left join task3_materialized t2 on t2.link2 = t1.link1
left join task39 t9 on t9.link9 = t2.link2
left join task35 t5 on t5.link5 = t1.link1
left join task36 t6 on t6.link6 = t1.link1
left join task310 t10 on t10.link10 = t1.link1
"""

libs.db_connection.select_data_dict_with_commit(sql1, [])

[RealDictRow([('QUERY PLAN',
               'Hash Right Join  (cost=337237.51..499431.66 rows=1352804 width=308) (actual time=5600.457..10578.025 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Hash Cond: (t2.link2 = t1.link1)')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Hash Left Join  (cost=42881.00..116566.10 rows=1352804 width=90) (actual time=749.167..2797.156 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN', '        Hash Cond: (t2.link2 = t9.link9)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Seq Scan on task3_materialized t2  (cost=0.00..24180.04 rows=1352804 width=36) (actual time=0.110..401.278 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Hash  (cost=20615.00..20615.00 rows=1000000 width=54) (actual time=748.744..748.746 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN',
               '              Buckets: 65536  Batches: 32  Memory Usage: 3171kB')]),
 RealDictRow([('QUERY PLAN',
     

In [2]:
# Переписанный запрос
sql1 = """explain (analyze) 
SELECT * 
FROM task31 t1
left join 
(
select t2.* 
from task32 t2 
left join task33 t3 on t3.link3 = t2.link2
left join task34 t4 on t4.link4 = t2.link2
left join task37 t7 on t7.link7 = t2.link2
left join task38 t8 on t8.link8 = t2.link2
left join task39 t9 on t9.link9 = t2.link2
)
t2 on t2.link2 = t1.link1
left join task35 t5 on t5.link5 = t1.link1
left join task36 t6 on t6.link6 = t1.link1
left join task310 t10 on t10.link10 = t1.link1;"""

libs.db_connection.select_data_dict_with_commit(sql1, [])


[RealDictRow([('QUERY PLAN',
               'Gather  (cost=270718.09..481710.21 rows=1000000 width=272) (actual time=15471.510..16716.134 rows=1000000 loops=1)')]),
 RealDictRow([('QUERY PLAN', '  Workers Planned: 2')]),
 RealDictRow([('QUERY PLAN', '  Workers Launched: 2')]),
 RealDictRow([('QUERY PLAN',
               '  ->  Parallel Hash Left Join  (cost=269718.09..380710.21 rows=416667 width=272) (actual time=15410.509..15654.551 rows=333333 loops=3)')]),
 RealDictRow([('QUERY PLAN', '        Hash Cond: (t1.link1 = t10.link10)')]),
 RealDictRow([('QUERY PLAN',
               '        ->  Parallel Hash Left Join  (cost=245656.09..324051.59 rows=416667 width=218) (actual time=13691.174..13999.628 rows=333333 loops=3)')]),
 RealDictRow([('QUERY PLAN',
               '              Hash Cond: (t1.link1 = t6.link6)')]),
 RealDictRow([('QUERY PLAN',
               '              ->  Parallel Hash Left Join  (cost=221591.09..273085.97 rows=416667 width=163) (actual time=11039.102..12303.7

In [ ]:
# Задание
# Посмотреть запрос и переписать его через промежуточную материализацию


sql1 = """explain (analyze)
SELECT * 
FROM task35 t5
left join task37 t7 on t7.link7 = t5.link5
left join task36 t6 on t6.link6 = t7.link7
;"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])


In [ ]:
# Результат

sql1 = """
explain (analyze)
select *
from
"""

            
libs.db_connection.select_data_dict_with_commit(sql1, [])

In [ ]:
# Переписанный запрос
sql1 = """

"""

libs.db_connection.select_data_dict_with_commit(sql1, [])